## A simple swarm

In [112]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
from core.agent import *
from core.world import * 
from core.resource import *
import numpy as np
from core.render import * 

In [114]:
world = World(dims = (100, 100),
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
swarm = [Agent() for i in range(1000)]
for agent in swarm:
    world.add_agent(agent)

In [115]:
swarm = initialize_positions_randomly(world, swarm)

In [116]:
# Test movements by doing random actions
def update():
    for agent in swarm:
        choice = np.random.randint(1, 5)
        agent.move(choice)

    world.update()

update()

In [117]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [118]:
# Validate the observation space provided is sensible
obs = swarm[0].get_observation()

print(obs.nearby_agents, obs.nearby_agents.shape)

[[  0   0 748   0   0   0   0]
 [  0   0 115   0   0   0   0]
 [  0   0   0   0   0 390   0]
 [  0   0 905   0   0   0   0]
 [  0   0 311   0   0   0 428]
 [  0   0  39   0   0   0   0]
 [  0   0   0   0   0   0   0]] (7, 7)


In [119]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.get_position()
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.get_agents()))

989 1000
